In [111]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [112]:
sentences_list = []

file1 = open('house_mouse.txt', 'r')
sentences1 = file1.readlines()
  
for sentence in sentences1:
    sentences_list.append(sentence.strip())

file1 = open('computer_mouse.txt', 'r')
sentences2 = file1.readlines()

for sentence in sentences2:
    sentences_list.append(sentence.strip())

In [137]:
tokenized_sentences = []

for sentence in sentences_list:
    marked_text = "[CLS] " + sentence + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    tokenized_sentences.append(tokenized_text)

print(tokenized_sentences[0])

['[CLS]', 'the', 'house', 'mouse', 'is', 'a', 'small', 'mammal', 'of', 'the', 'order', 'rode', '##nti', '##a', ',', 'characteristic', '##ally', 'having', 'a', 'pointed', 'snout', ',', 'large', 'rounded', 'ears', ',', 'and', 'a', 'long', 'and', 'almost', 'hair', '##less', 'tail', '.', '[SEP]']


In [138]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentences[0])
segments_ids = [1] * len(tokenized_sentences[0])

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
model.eval()

with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# hidden_states has 4 dimensions:
# 1. The layer number (13 layers)
# 2. The batch number (sentences)
# 3. The word / token number (tokens in sentence)
# 4. The hidden unit / feature number (768 features) 

In [139]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings[12][0][3][:16]

tensor([-0.9539,  0.2430, -1.0363, -0.5150,  1.7840,  0.1482,  0.9718,  0.5997,
         0.4075,  0.0578,  0.1220,  0.2515, -0.3121, -0.0157, -0.7274, -0.1088])